# What's happening here?
In this nootebook I will show you simple basic idea so far that gives rmse 18.6  
What I've done before and it didn't work out:  
- Use metadata and score data in two heads of NN (the idea is to help backbone coefficients change to right way to predict score)
    - Use MSE for score and BCE for meta
- Tried to create NN for dividing score to 3 parts (0-33, 34-66, 67-100) and then learn 3 different NN for predicting score in this intervals.
    - The problem was that it's not much difference between the pictures, so it can't be divided into score groups in efficient way.  

Searching public notebooks we can see this way of solving the problem:
- Normalize score from 0 to 1
- Dont use metadata
- Use vision transformer as base NN
- Use BCE as loss function

In [ ]:
import torch
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

import sys
sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')
import timm

from sklearn.model_selection import train_test_split

from PIL import Image
import glob
import os
import cv2

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

import torch.nn.functional as F
import torch.nn as nn
import torchvision
from torch.utils.data import DataLoader, Dataset

In [ ]:
df = pd.read_csv('../input/petfinder-pawpularity-score/train.csv')
# val = pd.read_csv('../input/petfinder-pawpularity-score/test.csv')
df['Pawpularity'] = df['Pawpularity']/100

### But I wanted to do some simple EDA and what I've find out

In [ ]:
df.groupby('Pawpularity').count().plot.bar(y='Id', figsize=(15,10), xticks=np.linspace(0,100,51), rot=0, xlabel='Pawpularity score', ylabel='Counts', \
                                           title='Distribution of scores by sample counts');

### Here above we can see a big bias in score of 100. Let's see if it's images are different form other scores

In [ ]:
paths100 = []
for _id in df[df['Pawpularity'] == 1]['Id']:
    paths100.append(os.path.join('../input/petfinder-pawpularity-score/train', _id + '.jpg'))

fig, axes = plt.subplots(5,5,figsize=(10,10))
fig.tight_layout()
for p, ax in zip(np.random.choice(paths100, size=5*5), axes.ravel()):
    image = cv2.imread(p, cv2.IMREAD_COLOR)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
    image /= 255.0
    ax.imshow(image)
    ax.axis('off')
axes.ravel()[2].set_title('Score 100');

In [ ]:
pathsnot100 = []
for _id in df[df['Pawpularity'] != 1]['Id']:
    pathsnot100.append(os.path.join('../input/petfinder-pawpularity-score/train', _id + '.jpg'))

fig, axes = plt.subplots(5,5,figsize=(10,10))
fig.tight_layout()
for p, ax in zip(np.random.choice(pathsnot100, size=5*5), axes.ravel()):
    image = cv2.imread(p, cv2.IMREAD_COLOR)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
    image /= 255.0
    ax.imshow(image)
    ax.axis('off')
axes.ravel()[2].set_title('Score not 100');

### As I expected I can't see particular difference between them. But when we will do train/test split we will use this information

In [ ]:
train, val = train_test_split(df, test_size=0.2, stratify=df['Pawpularity']==1.0)

In [ ]:
train.head()

In [ ]:
class PetDataset(Dataset):
    def __init__(self, dataframe, transforms=None):
        self.dataframe = dataframe
        self.transforms = transforms
        self.path = '../input/petfinder-pawpularity-score/train'

    def __getitem__(self, index):
        
        image_path = os.path.join(self.path, self.dataframe.iloc[index].Id + '.jpg')
        target = self.dataframe.iloc[index].Pawpularity
        
        # meta = np.array(self.dataframe.iloc[index,1:-1], dtype=np.int8)
        
        image = cv2.imread(image_path, cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0
        
        if self.transforms:
            image = self.transforms(image=image)['image']
        else:
            image = torch.tensor(image, dtype=torch.float32)
        targt = torch.as_tensor(target, dtype=torch.float32)
        # meta = torch.as_tensor(meta, dtype=torch.float32)
        # targt = torch.as_tensor(np.hstack((meta,target)), dtype=torch.float32)
        return image, targt
        
    def __len__(self):
        return len(self.dataframe)

In [ ]:
train_transform = A.Compose(
    [
        A.Resize(224, 224),# 224
        A.ShiftScaleRotate(shift_limit=0.2, scale_limit=0.2, rotate_limit=20, p=0.5),
        A.RGBShift(r_shift_limit=0.05, g_shift_limit=0.05, b_shift_limit=0.05, p=0.5),
        A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ToTensorV2(),
    ]
)

val_transform = A.Compose(
    [
        A.Resize(224, 224),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ToTensorV2(),
    ]
)

train_dataset = PetDataset(train, train_transform)
val_dataset = PetDataset(val, val_transform)

In [ ]:
train_loader = DataLoader(
    train_dataset,
    batch_size=16,
    shuffle=True,
    pin_memory=True,
)
val_loader = DataLoader(
    val_dataset,
    batch_size=8,
    shuffle=False,
    pin_memory=True,
)

In [ ]:
dataset = PetDataset(train)
def visualize(dataset):
    fig, ax = plt.subplots(nrows=4, ncols=4, figsize=(10,10))
    axes = ax.ravel()
    for i, (X, y) in enumerate(dataset):
        if i < len(axes):
            axes[i].imshow(X)
            axes[i].set_title(y)
            axes[i].axis('off')
        else:
            break

visualize(dataset)

### After many tries I found out good LR 

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = timm.create_model('swin_large_patch4_window7_224', pretrained=True, num_classes=1).to(device)

# pretrain head first for faster convergence
for p in model.parameters():
    p.requires_grad_(False)

for p in model.head.parameters():
    p.requires_grad_(True)

criterion = torch.nn.BCEWithLogitsLoss()

optimizer = torch.optim.AdamW(model.parameters(), lr=2e-3)
milestones = [1,2,3]
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=milestones, gamma=0.1, verbose=True)
scheduler_reduce = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3, verbose=True)
num_epochs = 20

In [ ]:
train_loader_len = len(train_loader)
val_loader_len = len(val_loader)
best_rmse = 1000
PAT = 5

for epoch in range(1, num_epochs + 1):
    
    model.train()
    closs = 0
    
    for X, y in train_loader:
        X, y = X.to(device), y.to(device).view(-1,1)
        model_output = model(X)
        loss = criterion(model_output, y)
        closs += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    print(f"Epoch {epoch}\nLoss {closs/train_loader_len:0.5f}")
    
    model.eval()
    crmse = 0
    with torch.no_grad():
        for X, y in val_loader:
            X, y = X.to(device), y.to(device).view(-1,1)
            model_output = model(X)
            rmse = 100*torch.sqrt(F.mse_loss(torch.sigmoid(model_output), y))
            crmse += rmse.item()
            
    mean_rmse = crmse/val_loader_len
    print(f"Mean rmse {mean_rmse:0.5f}\n")
    
    scheduler.step()
    scheduler_reduce.step(mean_rmse)
    
    if epoch == milestones[-1]:
        print('Unfreeze all')
        for p in model.parameters():
            p.requires_grad_(True)
    if mean_rmse <= best_rmse:
        patience = 0
        best_rmse = mean_rmse       
        print(f"Save with best_rmse {best_rmse}")
        torch.save(model.state_dict(), 'my_model.pth')
    else:
        if patience >= PAT:
            break
        else:
            patience += 1

#### What to do next?  
#### I think that key for better score is ensembling methods of this idea

# PLEASE UPVOTE THIS NOTEBOOK IF IT WAS USEFUL FOR YOU :)